In [7]:
%matplotlib inline
import warnings
warnings.filterwarnings(action='ignore')

# data
import pandas as pd
import numpy as np
import random as rnd
from sklearn.model_selection import KFold
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import train_test_split
from imblearn.over_sampling import SMOTE
from imblearn.under_sampling import RandomUnderSampler
from imblearn.pipeline import Pipeline
from collections import Counter

# visualization
import seaborn as sns
import matplotlib.pyplot as plt
%matplotlib inline

# machine learning
from sklearn.linear_model import LogisticRegression
from sklearn.svm import LinearSVR
from sklearn.ensemble import RandomForestRegressor
from sklearn.neighbors import KNeighborsClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.linear_model import Perceptron
from sklearn.linear_model import SGDClassifier
from sklearn.tree import DecisionTreeRegressor
from sklearn import metrics
import tensorflow as tf

from sklearn.model_selection import GridSearchCV
from sklearn.metrics import explained_variance_score
from sklearn.metrics import mean_absolute_error
from sklearn.metrics import mean_squared_error
from sklearn.metrics import r2_score

In [8]:
pd.set_option('display.max_columns', 200)
pd.set_option('display.max_rows', 200)

In [9]:
origin_df = pd.read_csv('bdb.csv', low_memory = False)
# origin_df.head(60)

In [10]:
origin_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 682154 entries, 0 to 682153
Data columns (total 49 columns):
GameId                    682154 non-null int64
PlayId                    682154 non-null int64
Team                      682154 non-null object
X                         682154 non-null float64
Y                         682154 non-null float64
S                         682154 non-null float64
A                         682154 non-null float64
Dis                       682154 non-null float64
Orientation               682131 non-null float64
Dir                       682126 non-null float64
NflId                     682154 non-null int64
DisplayName               682154 non-null object
JerseyNumber              682154 non-null int64
Season                    682154 non-null int64
YardLine                  682154 non-null int64
Quarter                   682154 non-null int64
GameClock                 682154 non-null object
PossessionTeam            682154 non-null object
Down   

# INVOLVED STUFF

In [ ]:
make Home column, binary values, use fr new team columns
# Team - home or away

point differential, plain value then positive o negative
# HomeScoreBeforePlay - home team score before play started
# VisitorScoreBeforePlay - visitor team score before play started

the whole shebang here:
# PossessionTeam - team with possession
# HomeTeamAbbr - home team abbreviation
# VisitorTeamAbbr - visitor team abbreviation

add isCarrier column
# NflIdRusher - the NflId of the rushing player

big ass block of fixing
# OffensePersonnel - offensive team positional grouping

fix needed then label encode
# Position - the player's position (the specific role on the field that they typically play)
# StadiumType - description of the stadium environment


# X - player position along the long axis of the field. See figure below.
# Y - player position along the short axis of the field. See figure below.

# S - speed in yards/second

# A - acceleration in yards/second^2

# Dis - distance traveled from prior time point, in yards

# Orientation - orientation of player (deg)
# FieldPosition - which side of the field the play is happening on
# Dir - angle of player motion (deg)
# PlayDirection - direction the play is headed

change to distance to endzone
# YardLine - the yard line of the line of scrimmage


# Down - the down (1-4)
# Distance - yards needed for a first down



# DefendersInTheBox - number of defenders lined up near the line of scrimmage, spanning the width of the offensive line
# DefensePersonnel - defensive team positional grouping

# TimeHandoff - UTC time of the handoff
# TimeSnap - UTC time of the snap
time frm snap to handoff, deltee these

# Yards - the yardage gained on the play (you are predicting this)
yard above average?


# Week - week into the season
percent of season over?

# GameWeather - description of the game weather
dummy values, o scaled


# EASY STUFF

In [ ]:
delete right away
# GameId - a unique game identifier
# Season - year of the season
# DisplayName - player's name
# JerseyNumber - jersey number
# PlayerBirthDate - birth date (mm/dd/yyyy)
# PlayerCollegeName - where the player attended college
# Stadium - stadium where the game is being played
# Location - city where the game is being played
# WindSpeed - wind speed in miles/hour
# WindDirection - wind direction

just check to make sure its good
# Temperature - temperature (deg F)
# Humidity - humidity

encode: grass, turf, hybrid    
# Turf - description of the field surface

leave how it came
# PlayerWeight - player weight (lbs)

convert to inches
# PlayerHeight - player height (ft-in)

keep but exclude frm model
# PlayId - a unique play identifier
# NflId - a unique identifier of the player

convert to seconds
# GameClock - time on the game clock quarter

!!! DECIDE
# Quarter - game quarter (1-5, 5 == overtime)

clean then label encode
# OffenseFormation - offense formation

In [11]:
working_df = origin_df.copy()

In [12]:
players = working_df[['NflId', 'DisplayName', 'JerseyNumber', 'PlayerBirthDate', 'PlayerCollegeName', 'Position']]

In [13]:
working_df.drop(columns=['DisplayName', 'JerseyNumber', 'PlayerBirthDate', 'PlayerCollegeName',
                        'Stadium', 'Location', 'WindSpeed', 'WindDirection', 'GameId', 'Season',
                        ], axis= 1, inplace= True)

In [14]:
working_df

,PlayId,Team,X,Y,S,A,Dis,Orientation,Dir,NflId,YardLine,Quarter,GameClock,PossessionTeam,Down,Distance,FieldPosition,HomeScoreBeforePlay,VisitorScoreBeforePlay,NflIdRusher,OffenseFormation,OffensePersonnel,DefendersInTheBox,DefensePersonnel,PlayDirection,TimeHandoff,TimeSnap,Yards,PlayerHeight,PlayerWeight,Position,HomeTeamAbbr,VisitorTeamAbbr,Week,StadiumType,Turf,GameWeather,Temperature,Humidity
0,20170907000118,away,73.91,34.84,1.69,1.13,0.40,81.99,177.18,496723,35,1,14:14:00,NE,3,2,NE,0,0,2543773,SHOTGUN,"1 RB, 1 TE, 3 WR",6.0,"2 DL, 3 LB, 6 DB",left,2017-09-08T00:44:06.000Z,2017-09-08T00:44:05.000Z,8,6-0,212,SS,NE,KC,1,Outdoor,Field Turf,Clear and warm,63.0,77.0
1,20170907000118,away,74.67,32.64,0.42,1.35,0.01,27.61,198.70,2495116,35,1,14:14:00,NE,3,2,NE,0,0,2543773,SHOTGUN,"1 RB, 1 TE, 3 WR",6.0,"2 DL, 3 LB, 6 DB",left,2017-09-08T00:44:06.000Z,2017-09-08T00:44:05.000Z,8,6-3,288,DE,NE,KC,1,Outdoor,Field Turf,Clear and warm,63.0,77.0
2,20170907000118,away,74.00,33.20,1.22,0.59,0.31,3.01,202.73,2495493,35,1,14:14:00,NE,3,2,NE,0,0,2543773,SHOTGUN,"1 RB, 1 TE, 3 WR",6.0,"2 DL, 3 LB, 6 DB",left,2017-09-08T00:44:06.000Z,2017-09-08T00:44:05.000Z,8,6-3,270,DE,NE,KC,1,Outdoor,Field Turf,Clear and warm,63.0,77.0
3,20170907000118,away,71.46,27.70,0.42,0.54,0.02,359.77,105.64,2506353,35,1,14:14:00,NE,3,2,NE,0,0,2543773,SHOTGUN,"1 RB, 1 TE, 3 WR",6.0,"2 DL, 3 LB, 6 DB",left,2017-09-08T00:44:06.000Z,2017-09-08T00:44:05.000Z,8,6-3,245,ILB,NE,KC,1,Outdoor,Field Turf,Clear and warm,63.0,77.0
4,20170907000118,away,69.32,35.42,1.82,2.43,0.16,12.63,164.31,2530794,35,1,14:14:00,NE,3,2,NE,0,0,2543773,SHOTGUN,"1 RB, 1 TE, 3 WR",6.0,"2 DL, 3 LB, 6 DB",left,2017-09-08T00:44:06.000Z,2017-09-08T00:44:05.000Z,8,6-0,206,FS,NE,KC,1,Outdoor,Field Turf,Clear and warm,63.0,77.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
682149,20191125003789,home,69.58,31.33,4.19,1.33,0.43,92.32,20.94,2556593,38,4,04:11:00,BLT,2,9,BLT,6,45,2562407,PISTOL,"1 RB, 1 TE, 3 WR",6.0,"3 DL, 3 LB, 5 DB",left,2019-11-26T03:59:06.000Z,2019-11-26T03:59:05.000Z,4,6-3,228,ILB,LA,BAL,12,Outdoor,Grass,Clear,62.0,64.0
682150,20191125003789,home,72.16,25.09,1.01,1.40,0.08,65.83,76.48,2558199,38,4,04:11:00,BLT,2,9,BLT,6,45,2562407,PISTOL,"1 RB, 1 TE, 3 WR",6.0,"3 DL, 3 LB, 5 DB",left,2019-11-26T03:59:06.000Z,2019-11-26T03:59:05.000Z,4,6-1,295,DT,LA,BAL,12,Outdoor,Grass,Clear,62.0,64.0
682151,20191125003789,home,72.23,31.95,1.75,0.68,0.17,93.99,90.26,2558830,38,4,04:11:00,BLT,2,9,BLT,6,45,2562407,PISTOL,"1 RB, 1 TE, 3 WR",6.0,"3 DL, 3 LB, 5 DB",left,2019-11-26T03:59:06.000Z,2019-11-26T03:59:05.000Z,4,6-3,245,OLB,LA,BAL,12,Outdoor,Grass,Clear,62.0,64.0
682152,20191125003789,home,72.08,26.74,0.70,1.59,0.07,61.47,48.34,2562391,38,4,04:11:00,BLT,2,9,BLT,6,45,2562407,PISTOL,"1 RB, 1 TE, 3 WR",6.0,"3 DL, 3 LB, 5 DB",left,2019-11-26T03:59:06.000Z,2019-11-26T03:59:05.000Z,4,6-1,312,DT,LA,BAL,12,Outdoor,Grass,Clear,62.0,64.0


# Offense Personnel

In [ ]:
# FIX GROUPS

### these we can make assumption of 1 QB and 5 OL, add both:

# 1 RB, 1 TE, 3 WR                310288 -- 1 QB, 5 OL, 1 RB, 1 TE, 3 WR
# 1 RB, 2 TE, 2 WR                155232 -- 1 QB, 5 OL, 1 RB, 2 TE, 2 WR 
# 2 RB, 1 TE, 2 WR                 71764 -- 1 QB, 5 OL, 2 RB, 1 TE, 2 WR 
# 1 RB, 3 TE, 1 WR                 39556 -- 1 QB, 5 OL, 1 RB, 3 TE, 1 WR 
# 2 RB, 2 TE, 1 WR                 34958 -- 1 QB, 5 OL, 2 RB, 2 TE, 1 WR
# 2 RB, 0 TE, 3 WR                  4752 -- 1 QB, 5 OL, 2 RB, 0 TE, 3 WR 
# 1 RB, 0 TE, 4 WR                  3828 -- 1 QB, 5 OL, 1 RB, 0 TE, 4 WR
# 2 RB, 3 TE, 0 WR                  2134 -- 1 QB, 5 OL, 2 RB, 3 TE, 0 WR
# 3 RB, 1 TE, 1 WR                   726 -- 1 QB, 5 OL, 3 RB, 1 TE, 1 WR 
# 3 RB, 0 TE, 2 WR                   308 -- 1 QB, 5 OL, 3 RB, 0 TE, 2 WR 
# 0 RB, 2 TE, 3 WR                   220 -- 1 QB, 5 OL, 0 RB, 2 TE, 3 WR 
# 1 RB, 4 TE, 0 WR                   154 -- 1 QB, 5 OL, 1 RB, 4 TE, 0 WR 
# 3 RB, 2 TE, 0 WR                   132 -- 1 QB, 5 OL, 3 RB, 2 TE, 0 WR 
# 0 RB, 3 TE, 2 WR                    66 -- 1 QB, 5 OL, 0 RB, 3 TE, 2 WR 
# 0 RB, 0 TE, 5 WR                    44 -- 1 QB, 5 OL, 0 RB, 0 TE, 5 WR 
# 0 RB, 1 TE, 4 WR                  1738 -- 1 QB, 5 OL, 0 RB, 1 TE, 4 WR 

working_df['OffensePersonnel'].replace(to_replace={'1 RB, 1 TE, 3 WR' : '1 QB, 5 OL, 1 RB, 1 TE, 3 WR', 
                                                   '1 RB, 2 TE, 2 WR' : '1 QB, 5 OL, 1 RB, 2 TE, 2 WR',  
                                                   '2 RB, 1 TE, 2 WR' : '1 QB, 5 OL, 2 RB, 1 TE, 2 WR',  
                                                   '1 RB, 3 TE, 1 WR' : '1 QB, 5 OL, 1 RB, 3 TE, 1 WR',  
                                                   '2 RB, 2 TE, 1 WR' : '1 QB, 5 OL, 2 RB, 2 TE, 1 WR', 
                                                   '2 RB, 0 TE, 3 WR' : '1 QB, 5 OL, 2 RB, 0 TE, 3 WR',  
                                                   '1 RB, 0 TE, 4 WR' : '1 QB, 5 OL, 1 RB, 0 TE, 4 WR', 
                                                   '2 RB, 3 TE, 0 WR' : '1 QB, 5 OL, 2 RB, 3 TE, 0 WR', 
                                                   '3 RB, 1 TE, 1 WR' : '1 QB, 5 OL, 3 RB, 1 TE, 1 WR',  
                                                   '3 RB, 0 TE, 2 WR' : '1 QB, 5 OL, 3 RB, 0 TE, 2 WR',  
                                                   '0 RB, 2 TE, 3 WR' : '1 QB, 5 OL, 0 RB, 2 TE, 3 WR',  
                                                   '1 RB, 4 TE, 0 WR' : '1 QB, 5 OL, 1 RB, 4 TE, 0 WR',  
                                                   '3 RB, 2 TE, 0 WR' : '1 QB, 5 OL, 3 RB, 2 TE, 0 WR',  
                                                   '0 RB, 3 TE, 2 WR' : '1 QB, 5 OL, 0 RB, 3 TE, 2 WR',  
                                                   '0 RB, 0 TE, 5 WR' : '1 QB, 5 OL, 0 RB, 0 TE, 5 WR',
                                                   '0 RB, 1 TE, 4 WR' : '1 QB, 5 OL, 0 RB, 1 TE, 4 WR'
                                                  }, inplace = True)
                                       

### these we can make assumption of 1 QB and 6 OL, add QB:

# 6 OL, 3 RB, 0 TE, 1 WR              22 -- 1 QB, 6 OL, 3 RB, 0 TE, 1 WR 
# 6 OL, 0 RB, 2 TE, 2 WR              22 -- 1 QB, 6 OL, 0 RB, 2 TE, 2 WR 
# 6 OL, 2 RB, 0 TE, 2 WR            1452 -- 1 QB, 6 OL, 2 RB, 0 TE, 2 WR
# 6 OL, 1 RB, 3 TE, 0 WR            1606 -- 1 QB, 6 OL, 1 RB, 3 TE, 0 WR
# 6 OL, 1 RB, 0 TE, 3 WR            2948 -- 1 QB, 6 OL, 1 RB, 0 TE, 3 WR
# 6 OL, 2 RB, 2 TE, 0 WR            4444 -- 1 QB, 6 OL, 2 RB, 2 TE, 0 WR

# 6 OL, 2 RB, 1 TE, 1 WR            7392 -- 1 QB, 6 OL, 2 RB, 1 TE, 1 WR
# 6 OL, 1 RB, 1 TE, 2 WR           12936 -- 1 QB, 6 OL, 1 RB, 1 TE, 2 WR
# 6 OL, 1 RB, 2 TE, 1 WR           14190 -- 1 QB, 6 OL, 1 RB, 2 TE, 1 WR

working_df['OffensePersonnel'].replace(to_replace={'6 OL, 3 RB, 0 TE, 1 WR' : '1 QB, 6 OL, 3 RB, 0 TE, 1 WR', 
                                                   '6 OL, 0 RB, 2 TE, 2 WR' : '1 QB, 6 OL, 0 RB, 2 TE, 2 WR', 
                                                   '6 OL, 2 RB, 0 TE, 2 WR' : '1 QB, 6 OL, 2 RB, 0 TE, 2 WR', 
                                                   '6 OL, 1 RB, 3 TE, 0 WR' : '1 QB, 6 OL, 1 RB, 3 TE, 0 WR', 
                                                   '6 OL, 1 RB, 0 TE, 3 WR' : '1 QB, 6 OL, 1 RB, 0 TE, 3 WR',
                                                   '6 OL, 2 RB, 2 TE, 0 WR' : '1 QB, 6 OL, 1 RB, 0 TE, 3 WR', 
                                                   '6 OL, 2 RB, 1 TE, 1 WR' : '1 QB, 6 OL, 2 RB, 1 TE, 1 WR', 
                                                   '6 OL, 1 RB, 1 TE, 2 WR' : '1 QB, 6 OL, 1 RB, 1 TE, 2 WR', 
                                                   '6 OL, 1 RB, 2 TE, 1 WR' : '1 QB, 6 OL, 1 RB, 2 TE, 1 WR'
                                                  }, inplace= True)


### due to high count of plays, it is unlikely 7 ol were used. RB was probs seen as OL. 
### subtract one OL, add on RB, add QB

# 7 OL, 1 RB, 0 TE, 2 WR             682 -- 1 QB, 6 OL, 2 RB, 0 TE, 2 WR
# 7 OL, 1 RB, 2 TE, 0 WR             308 -- 1 QB, 6 OL, 2 RB, 2 TE, 0 WR
# 7 OL, 1 RB, 1 TE, 1 WR              44 -- 1 QB, 6 OL, 1 RB, 2 TE, 1 WR
# 7 OL, 2 RB, 1 TE, 0 WR              88 -- 1 QB, 6 OL, 3 RB, 1 TE, 0 WR
# 7 OL, 2 RB, 0 TE, 1 WR             220 -- 1 QB, 6 OL, 3 RB, 0 TE, 1 WR
# 7 OL, 1 RB, 2 TE, 0 WR             308 -- 1 QB, 6 OL, 2 RB, 2 TE, 0 WR

working_df['OffensePersonnel'].replace(to_replace={'7 OL, 1 RB, 0 TE, 2 WR' : '1 QB, 6 OL, 2 RB, 0 TE, 2 WR', 
                                                   '7 OL, 1 RB, 2 TE, 0 WR' : '1 QB, 6 OL, 2 RB, 2 TE, 0 WR', 
                                                   '7 OL, 1 RB, 1 TE, 1 WR' : '1 QB, 6 OL, 1 RB, 2 TE, 1 WR', 
                                                   '7 OL, 2 RB, 1 TE, 0 WR' : '1 QB, 6 OL, 3 RB, 1 TE, 0 WR',
                                                   '7 OL, 2 RB, 0 TE, 1 WR' : '1 QB, 6 OL, 3 RB, 0 TE, 1 WR',
                                                   '7 OL, 1 RB, 2 TE, 0 WR' : '1 QB, 6 OL, 2 RB, 2 TE, 0 WR'
                                                  }, inplace= True)

### the following were corrected by verifying actual game film for each play (NFL GamePass)
### play numbers are included followed by correct grouping and ind. replacement code:

# 1 RB, 1 TE, 2 WR,1 LB               66 -- 1 QB, 5 OL, 2 RB, 1 TE, 2 WR -- 20171113000112 / 20171113001385 / 20191117101942
# 1 RB, 0 TE, 3 WR,1 DB               66 -- 1 QB, 5 OL, 1 RB, 0 TE, 4 WR -- 20171105060104 / 20171112061099 / 20171112063586
# 2 QB, 2 RB, 0 TE, 2 WR              66 -- 1 QB, 5 OL, 3 RB, 0 TE, 2 WR -- 20181021012259 / 20181209042583 / 20191117063033
# 1 RB, 2 TE, 1 WR,1 DB               66 -- 1 QB, 5 OL, 1 RB, 2 TE, 2 WR -- 20181111110273 / 20181118002325 / 20181129000675
# 2 QB, 1 RB, 3 TE, 0 WR              44 -- 1 QB, 5 OL, 2 RB, 3 TE, 0 WR -- 20181028013051 / 20181028112146
# 6 OL, 1 RB, 0 TE, 2 WR,1 LB         44 -- 1 QB, 6 OL, 2 RB, 0 TE, 2 WR -- 20191117100236 / 20191117100497
# 2 RB, 2 TE, 0 WR,1 DL               44 -- 1 QB, 6 OL, 2 RB, 2 TE, 0 WR -- 20180930100710 / 20181101000953
# 7 OL, 1 RB, 1 TE, 0 WR,1 LB         44 -- 1 QB, 6 OL, 3 RB, 1 TE, 0 WR -- 20190915090924 / 20190915090949
# 2 QB, 2 RB, 2 TE, 0 WR              22 -- 1 QB, 5 OL, 3 RB, 2 TE, 0 WR -- 20181008003713
# 2 RB, 3 TE, 1 WR                    22 -- 1 QB, 5 OL, 1 RB, 3 TE, 1 WR -- 20170924102908
# 1 RB, 3 TE, 0 WR,1 DB               22 -- 1 QB, 5 OL, 1 RB, 3 TE, 1 WR -- 20171116000514
# 6 OL, 1 RB, 1 TE, 1 WR,1 LB         22 -- 1 QB, 6 OL, 2 RB, 1 TE, 1 WR -- 20191124090534
# 1 RB, 2 TE, 3 WR                    22 -- 1 QB, 5 OL, 1 RB, 2 TE, 2 WR -- 20171112080136
# 2 RB, 1 TE, 1 WR,1 DB               22 -- 1 QB, 5 OL, 2 RB, 1 TE, 2 WR -- 20181104013118
# 6 OL, 1 RB, 1 TE, 0 WR,2 DL         22 -- 1 QB, 6 OL, 3 RB, 1 TE, 0 WR -- 20181202071749
# 2 QB, 3 RB, 1 TE, 0 WR              22 -- 1 QB, 6 OL, 3 RB, 1 TE, 0 WR -- 20181021012713

working_df['OffensePersonnel'].replace(to_replace={'1 RB, 1 TE, 2 WR,1 LB' : '1 QB, 5 OL, 2 RB, 1 TE, 2 WR', 
                                                   '1 RB, 0 TE, 3 WR,1 DB' : '1 QB, 5 OL, 1 RB, 0 TE, 4 WR', 
                                                   '2 QB, 2 RB, 0 TE, 2 WR' : '1 QB, 5 OL, 3 RB, 0 TE, 2 WR', 
                                                   '1 RB, 2 TE, 1 WR,1 DB' : '1 QB, 5 OL, 1 RB, 2 TE, 2 WR', 
                                                   '2 QB, 1 RB, 3 TE, 0 WR' : '1 QB, 5 OL, 2 RB, 3 TE, 0 WR', 
                                                   '6 OL, 1 RB, 0 TE, 2 WR,1 LB' : '1 QB, 6 OL, 2 RB, 0 TE, 2 WR', 
                                                   '2 RB, 2 TE, 0 WR,1 DL' : '1 QB, 6 OL, 2 RB, 2 TE, 0 WR', 
                                                   '7 OL, 1 RB, 1 TE, 0 WR,1 LB' : '1 QB, 6 OL, 3 RB, 1 TE, 0 WR', 
                                                   '2 QB, 2 RB, 2 TE, 0 WR' : '1 QB, 5 OL, 3 RB, 2 TE, 0 WR', 
                                                   '2 RB, 3 TE, 1 WR' : '1 QB, 5 OL, 1 RB, 3 TE, 1 WR', 
                                                   '1 RB, 3 TE, 0 WR,1 DB' : '1 QB, 5 OL, 1 RB, 3 TE, 1 WR', 
                                                   '6 OL, 1 RB, 1 TE, 1 WR,1 LB' : '1 QB, 6 OL, 2 RB, 1 TE, 1 WR', 
                                                   '1 RB, 2 TE, 3 WR' : '1 QB, 5 OL, 1 RB, 2 TE, 2 WR', 
                                                   '2 RB, 1 TE, 1 WR,1 DB' : '1 QB, 5 OL, 2 RB, 1 TE, 2 WR', 
                                                   '6 OL, 1 RB, 1 TE, 0 WR,2 DL' : '1 QB, 6 OL, 3 RB, 1 TE, 0 WR', 
                                                   '2 QB, 3 RB, 1 TE, 0 WR' : '1 QB, 6 OL, 3 RB, 1 TE, 0 WR', 
                                                  }, inplace=True)


### finally, the following were adjusted using football logic
### when in doubt, an out of place player is counted as RB since this df is rushing plays
### whether Machine Vision or manual jersey number, DL are probs OL

# 1 RB, 2 TE, 1 WR,1 DL             3102 -- 1 QB, 6 OL, 1 RB, 2 TE, 1 WR
# 2 QB, 1 RB, 1 TE, 2 WR            1716 -- 1 QB, 5 OL, 2 RB, 1 TE, 2 WR
# 1 RB, 1 TE, 2 WR,1 DL              748 -- 1 QB, 6 OL, 1 RB, 1 TE, 2 WR
# 2 QB, 2 RB, 1 TE, 1 WR             550 -- 1 QB, 5 OL, 3 RB, 1 TE, 1 WR
# 1 RB, 3 TE, 0 WR,1 DL              506 -- 1 QB, 6 OL, 1 RB, 3 TE, 0 WR
# 2 QB, 1 RB, 2 TE, 1 WR             462 -- 1 QB, 5 OL, 2 RB, 2 TE, 1 WR
# 6 OL, 1 RB, 2 TE, 0 WR,1 LB        440 -- 1 QB, 6 OL, 2 RB, 2 TE, 0 WR
# 6 OL, 1 RB, 2 TE, 0 WR,1 DL        374 -- 1 QB, 6 OL, 2 RB, 2 TE, 0 WR
# 1 RB, 2 TE, 1 WR,1 LB              264 -- 1 QB, 5 OL, 2 RB, 2 TE, 1 WR
# 1 RB, 1 TE, 2 WR,1 DB              242 -- 1 QB, 5 OL, 1 RB, 1 TE, 3 WR
# 6 OL, 2 RB, 1 TE, 0 WR,1 DL        198 -- 1 QB, 6 OL, 3 RB, 1 TE, 0 WR
# 2 QB, 1 RB, 0 TE, 3 WR             198 -- 1 QB, 5 OL, 2 RB, 0 TE, 3 WR
# 2 QB, 6 OL, 1 RB, 1 TE, 1 WR       176 -- 1 QB, 6 OL, 2 RB, 1 TE, 1 WR
# 6 OL, 1 RB, 1 TE, 1 WR,1 DL        154 -- 1 QB, 6 OL, 2 RB, 1 TE, 1 WR
# 1 RB, 3 TE, 0 WR,1 LB              154 -- 1 QB, 5 OL, 2 RB, 3 TE, 0 WR

working_df['OffensePersonnel'].replace(to_replace={'1 RB, 2 TE, 1 WR,1 DL' : '1 QB, 6 OL, 1 RB, 2 TE, 1 WR', 
                                                   '2 QB, 1 RB, 1 TE, 2 WR' : '1 QB, 5 OL, 2 RB, 1 TE, 2 WR',
                                                   '1 RB, 1 TE, 2 WR,1 DL' : '1 QB, 6 OL, 1 RB, 1 TE, 2 WR', 
                                                   '2 QB, 2 RB, 1 TE, 1 WR' : '1 QB, 5 OL, 3 RB, 1 TE, 1 WR', 
                                                   '1 RB, 3 TE, 0 WR,1 DL' : '1 QB, 6 OL, 1 RB, 3 TE, 0 WR',
                                                   '2 QB, 1 RB, 2 TE, 1 WR' : '1 QB, 5 OL, 2 RB, 2 TE, 1 WR', 
                                                   '6 OL, 1 RB, 2 TE, 0 WR,1 LB' : '1 QB, 6 OL, 2 RB, 2 TE, 0 WR', 
                                                   '6 OL, 1 RB, 2 TE, 0 WR,1 DL' : '1 QB, 6 OL, 2 RB, 2 TE, 0 WR', 
                                                   '1 RB, 2 TE, 1 WR,1 LB' : '1 QB, 5 OL, 2 RB, 2 TE, 1 WR', 
                                                   '1 RB, 1 TE, 2 WR,1 DB' : '1 QB, 5 OL, 1 RB, 1 TE, 3 WR', 
                                                   '6 OL, 2 RB, 1 TE, 0 WR,1 DL' : '1 QB, 6 OL, 3 RB, 1 TE, 0 WR', 
                                                   '2 QB, 1 RB, 0 TE, 3 WR' : '1 QB, 5 OL, 2 RB, 0 TE, 3 WR', 
                                                   '2 QB, 6 OL, 1 RB, 1 TE, 1 WR' : '1 QB, 6 OL, 2 RB, 1 TE, 1 WR', 
                                                   '6 OL, 1 RB, 1 TE, 1 WR,1 DL' : '1 QB, 6 OL, 2 RB, 1 TE, 1 WR',
                                                   '1 RB, 3 TE, 0 WR,1 LB' : '1 QB, 5 OL, 2 RB, 3 TE, 0 WR'
                                                  }, inplace= True)

In [ ]:
# function needs to go to lh, removed from here 

def labelRB(OffensePersonnel):
    groups = OffensePersonnel.split(',')
    for group in groups:
        if "RB" in group:
            backs = [int(x) for x in group if x.isdigit()]
            backs = backs[0]
            return backs

working_df['RB'] = working_df['OffensePersonnel'].apply(labelRB)

In [ ]:
# function needs to go to lh, removed from here 

def labelTE(OffensePersonnel):
    groups = OffensePersonnel.split(',')
    for group in groups:
        if "TE" in group:
            ends = [int(x) for x in group if x.isdigit()]
            ends = ends[0]
            return ends
        
working_df['TE'] = working_df['OffensePersonnel'].apply(labelTE)

In [ ]:
# function needs to go to lh, removed from here 

def labelWR(OffensePersonnel):
    groups = OffensePersonnel.split(',')
    for group in groups:
        if "WR" in group:
            receivers = [int(x) for x in group if x.isdigit()]
            receivers = receivers[0]
            return receivers

working_df['WR'] = working_df['OffensePersonnel'].apply(labelWR)

In [ ]:
# function needs to go to lh, removed from here 

def labelQB(OffensePersonnel):
    groups = OffensePersonnel.split(',')
    for group in groups:
        if "QB" in group:
            passers = [int(x) for x in group if x.isdigit()]
            passers = passers[0]
            return passers

working_df['QB'] = working_df['OffensePersonnel'].apply(labelQB)

In [ ]:
# function needs to go to lh, removed from here 

def labelOL(OffensePersonnel):
    groups = OffensePersonnel.split(',')
    for group in groups:
        if "OL" in group:
            linemen = [int(x) for x in group if x.isdigit()]
            linemen = linemen[0]
            return linemen

working_df['OL'] = working_df['OffensePersonnel'].apply(labelOL)

In [ ]:
working_df.head()

## Team Names

In [ ]:
# I ran each of the below lines inidividually and could already
# see issues with teams like Arizona and Cleveland.
team_names = pd.DataFrame()
team_names['SortedPossession'] = sorted((working_df['PossessionTeam'].unique()))
team_names['SortedHome'] = sorted((working_df['HomeTeamAbbr'].unique()))
team_names['SortedVisitor'] = sorted((working_df['VisitorTeamAbbr'].unique()))

In [ ]:
# The below code fixes incorrect names and is included in the data
# cleaning module:

working_df['PossessionTeam'].replace(to_replace= {'ARZ':'ARI', 'BLT':'BAL',
                                                  'CLV':'CLE', 'HST':'HOU'}, 
                                    inplace= True)

working_df['FieldPosition'].replace(to_replace= {'ARZ':'ARI', 'BLT':'BAL',
                                                  'CLV':'CLE', 'HST':'HOU'}, 
                                    inplace= True)

In [ ]:
# rerun team_names to check fix
team_names

In [ ]:
# now that all names are fixed we have four name columns, HomeTeamAbbr,
# VisitorTeamAbbr, PossessionTeam, and Team. We'll create a table with this side by side.

team_columns = pd.DataFrame()
team_columns['PossessionTeam'] = working_df['PossessionTeam']
team_columns['HomeTeamAbbr'] = working_df['HomeTeamAbbr']
team_columns['VisitorTeamAbbr'] = working_df['VisitorTeamAbbr']
team_columns['Team'] = working_df['Team']

team_columns.head(10)

In [ ]:
working_df['offensiveTeam'] = working_df['PossessionTeam']
working_df.loc[working_df['PossessionTeam'] == working_df['HomeTeamAbbr'], 'defensiveTeam'] = working_df['VisitorTeamAbbr']
working_df.loc[working_df['PossessionTeam'] == working_df['VisitorTeamAbbr'], 'defensiveTeam'] = working_df['HomeTeamAbbr']

working_df.loc[working_df['Team'] == 'away', 'Team'] = working_df['VisitorTeamAbbr']
working_df.loc[working_df['Team'] == 'home', 'Team'] = working_df['HomeTeamAbbr']

# after running this, rerun team_columns cell above to check work

In [ ]:
team_columns['offensiveTeam'] = working_df['offensiveTeam']
team_columns['defensiveTeam'] = working_df['defensiveTeam']
team_columns.head()

In [ ]:
working_df.head()

In [ ]:
working_df.drop(columns= ['HomeTeamAbbr', 'VisitorTeamAbbr'], 
                axis=1, inplace=True)

# Turf Encoding

In [ ]:
working_df['Turf'].value_counts()

In [ ]:
# simplify Turf classes
turf_list = ['Field Turf', 'Artificial', 'FieldTurf', 'UBU Speed Series-S5-M',
            'A-Turf Titan', 'UBU Sports Speed S5-M', 'FieldTurf360', 'Twenty-Four/Seven Turf',
            'FieldTurf 360', 'Twenty Four/Seven Turf', 'Turf', 'Field turf', 
             'UBU-Speed Series-S5-M', 'Artifical']
grass_list = ['Grass', 'Natural Grass', 'Natural', 'Naturall Grass', 'natural grass', 'grass',
             'Natural grass']
hybrid_list = ['SISGrass', 'DD GrassMaster']

working_df.loc[working_df['Turf'].isin(turf_list), 'Turf'] = "turf"
working_df.loc[working_df['Turf'].isin(grass_list), 'Turf'] = "grass"
working_df.loc[working_df['Turf'].isin(hybrid_list), 'Turf'] = "hybrid"

In [ ]:
working_df['Turf'].value_counts()

In [ ]:
working_df.head()

# Team Assignments

In [ ]:
working_df['isOffense'] = 0
working_df['Opponent'] = None

working_df.loc[working_df['Team'] == working_df['offensiveTeam'], 'isOffense'] = 1 
working_df.loc[working_df['Team'] == working_df['defensiveTeam'], 'isOffense'] = 0 

working_df.loc[working_df['Team'] == working_df['offensiveTeam'], 'Opponent'] = working_df['defensiveTeam']
working_df.loc[working_df['Team'] == working_df['defensiveTeam'], 'Opponent'] = working_df['offensiveTeam'] 

In [ ]:
working_df.drop(columns=['offensiveTeam', 'defensiveTeam'], axis=1, inplace=True)

# Player Size

In [ ]:
# function needs to go to lh, removed from here 

def PlayerHeightInches(PlayerHeight):
    PlayerHeight.split('-')
    PlayerHeightInches = (int(PlayerHeight[0]) * 12) + int(PlayerHeight[2])
    return PlayerHeightInches

working_df['PlayerHeightInches'] = (working_df['PlayerHeight'].apply(PlayerHeightInches)).astype(int)

working_df.drop(columns=['PlayerHeight'], axis=1, inplace=True)

working_df.head()

# Yards

In [ ]:
averageYards = ((working_df['Yards'].mean()).round(0)).astype(int)
yards = working_df['Yards']

working_df['yardsAboveAverage'] = (yards) - averageYards

working_df.drop(columns=['Yards'], axis=1, inplace=True)

In [ ]:
working_df['yardsToEndzone'] = 0

working_df.loc[working_df['FieldPosition'] == working_df['NflId'], 'isCarrier'] = 1 
working_df.loc[working_df['FieldPosition'] != working_df['NflId'], 'isCarrier'] = 0   

# Game Clock

In [ ]:
# function needs to go to lh, removed from here 

# Because Quarter is a feature, we can just make convert this column into seconds that are left in the quarter

def GameClockSeconds(GameClock):
    time_split = GameClock.split(':')
    seconds = int(time_split[0])*60 + int(time_split[1]) 
    return seconds

working_df['GameClock'] = working_df['GameClock'].apply(GameClockSeconds)

In [ ]:
# function needs to go to lh, removed from here 

# used this to check if we even need 3rd part of GameClock split above- we don't

def last_of_gc(GameClock):
    time_split = GameClock.split(':')
    last = int(time_split[2])
    return last

In [ ]:
working_df

# Ball Carrier

In [ ]:
working_df['isCarrier'] = 0

working_df.loc[working_df['NflIdRusher'] == working_df['NflId'], 'isCarrier'] = 1 
working_df.loc[working_df['NflIdRusher'] != working_df['NflId'], 'isCarrier'] = 0 

In [ ]:
working_df.drop(columns=['PossessionTeam', 'VisitorScoreBeforePlay', 'NflIdRusher',
                        'OffensePersonnel', 'DefensePersonnel'], axis=1, inplace=True)

# Position

In [ ]:
working_df['Position'].value_counts()

In [ ]:
# simplify Position classes
db_list = ['CB', 'FS', 'SS', 'DB', 'S', 'SAF']
nonLineOffenseList = ['WR', 'TE', 'RB', 'QB', 'FB', 'HB']

working_df.loc[working_df['Position'].isin(db_list), 'Position'] = "DB"
working_df.loc[working_df['Position'].isin(nonLineOffenseList), 'Position'] = "OP"

working_df['Position'].value_counts()

In [ ]:
working_df['Position'].replace(to_replace= 'G', value='OG', inplace=True) 

In [ ]:
working_df.dtypes

# AFTER EDA AND CLEANING EXPERIMENTS, DO ASSUMPTION CHECKING BELOW: